In [115]:
from PIL import Image

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import imghdr

In [29]:
# Cleaning GPU memory.
tf.keras.backend.clear_session()

In [30]:
# Limiting gpu usage.
gpus = tf.config.list_physical_devices(device_type="GPU")
if gpus:
    for gpu in gpus:
        tf.config.set_logical_device_configuration(
            gpu,
            [tf.config.LogicalDeviceConfiguration(memory_limit=1024)]
        )

In [31]:
# Pre-trained model. (MobilenetV2)
preprocessing_data = tf.keras.applications.mobilenet_v2.preprocess_input
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False
)

In [27]:
# Preparing final layers.

input = tf.keras.Input(shape=(224,224,3))

preprocess = preprocessing_data(input)

x = base_model(preprocess, training=False)
x = tf.keras.layers.Flatten()(x)

bx = tf.keras.layers.Dense(units=128, activation="relu")(x)
bx = tf.keras.layers.Dense(units=128, activation="relu")(bx)
bx = tf.keras.layers.Dense(units=4)(bx)

cls = tf.keras.layers.Dense(units=64, activation="relu")(x)
cls = tf.keras.layers.Dense(units=64, activation="relu")(cls)
cls = tf.keras.layers.Dense(units=3, activation="softmax")(cls)

output = tf.keras.layers.Concatenate(axis=1)([cls, bx])

# Model.
model = tf.keras.Model(inputs=input, outputs=output)

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 7, 7, 1280)  2257984     ['input_2[0][0]']                
 al)                                                                                              
                                                                                                  
 flatten (Flatten)              (None, 62720)        0           ['mobilenetv2_1.00_224[0][0]']   
                                                                                              

### **Handle data**

In [140]:
def load_splice_data(dataset_path:str):

    if type(dataset_path)==str:
        assert os.path.isdir(dataset_path)
    else:
        raise
    
    images = []
    labels = []

    for dir in os.listdir(dataset_path):
        for data_dir in ["images", "labels"]:
            for data in os.listdir(os.path.join(dataset_path, dir, data_dir)):
                if data.endswith(".json"):
                    labels.append(os.path.join(dataset_path, dir, data_dir, data))
                elif data.endswith(".jpg"):
                    images.append(os.path.join(dataset_path, dir, data_dir, data))
    return np.array(images), np.array(labels)

In [141]:
DATASET_PATH = "/home/brain-matheus/BRAIN-Project/helmet_detector/Splice - motorcycleHelmet/motocicleta-dataset"

In [142]:
images, labels = load_splice_data(DATASET_PATH)

In [145]:
images

array(['/home/brain-matheus/BRAIN-Project/helmet_detector/Splice - motorcycleHelmet/motocicleta-dataset/Motocicleta-faixa1-noturno/Motocicleta/BE00015723000300337400Oeste1110320142129190600775783190009000.jpg',
       '/home/brain-matheus/BRAIN-Project/helmet_detector/Splice - motorcycleHelmet/motocicleta-dataset/Motocicleta-faixa1-noturno/Motocicleta/BE00015723000300337400Oeste1110320142004380600685774310009000.jpg',
       '/home/brain-matheus/BRAIN-Project/helmet_detector/Splice - motorcycleHelmet/motocicleta-dataset/Motocicleta-faixa1-noturno/Motocicleta/BE00015723000300337400Oeste1110320142222060600695789460009000.jpg',
       '/home/brain-matheus/BRAIN-Project/helmet_detector/Splice - motorcycleHelmet/motocicleta-dataset/Motocicleta-faixa1-noturno/Motocicleta/BE00015723000300337400Oeste2110320142109440600615781270009000.jpg',
       '/home/brain-matheus/BRAIN-Project/helmet_detector/Splice - motorcycleHelmet/motocicleta-dataset/Motocicleta-faixa1-noturno/Motocicleta/BE00015723000